<a href="https://colab.research.google.com/github/mission-impozzible/learning/blob/main/network_analysis/2025_11_26_network_analysis_community_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Network Analysis

1. Introduzione
1. Cos'è una rete a invarianza di scala?
1. Cosa si intende per reti orientate e il concetto di spazializzazione
1. Cos'è una ricevuta bancaria?
1. Spiegami l'algoritmo di Louvain
1. Spiegami l'algoritmo Spinglass
1. Quali alternative esistono?
1. Spiegami il concetto di modularità
1. Estraimi i principali insights da questo paper (Newman, 2006)

**References**  
Newman, M. E. (2006). Modularity and community structure in networks. Proceedings of the national academy of sciences, 103(23), 8577-8582.




# Community Detection

La **Community Detection** (o rilevazione di comunità) è un processo fondamentale nell'ambito della network analysis il cui scopo è **identificare raggruppamenti di nodi** all'interno di una rete complessa. Questi gruppi, noti come comunità o cluster, si distinguono perché i loro membri sono **più densamente interconnessi** tra loro rispetto ai nodi del resto della rete. L'obiettivo primario di questa analisi è svelare la **struttura a comunità intrinseca** o la modularità della rete. Tali sottostrutture riflettono spesso moduli significativi o unità funzionali nel mondo reale, come le cerchie sociali in un social network, i gruppi di ricerca che collaborano in una rete di co-autori, o i moduli di proteine interagenti in una rete biologica.

Le comunità sono definite in base al principio di connettività differenziale: i nodi appartenenti alla stessa comunità sono caratterizzati da **connessioni intra-comunità** molto più numerose e dense, che li legano fortemente l'uno all'altro. Al contrario, le **connessioni inter-comunità**, ovvero i legami tra nodi appartenenti a comunità diverse, sono relativamente **scarse e deboli**. Questa differenza nella densità di collegamento è il segnale che permette agli algoritmi di distinguere i confini tra i vari gruppi. La Community Detection è, in sintesi, uno strumento cruciale per la **riduzione della complessità** e per l'interpretazione dei dati nelle reti, offrendo una mappa più leggibile e significativa della loro organizzazione interna.


Poiché il numero e la dimensione delle comunità sono solitamente sconosciuti *a priori*, la loro identificazione è un problema computazionalmente difficile che viene affrontato con una varietà di metodi. Tra gli approcci più diffusi vi è l'**ottimizzazione della Modularità**, che impiega algoritmi come il Metodo Louvain per massimizzare una misura ($M$) che valuta la qualità della partizione confrontando i legami interni alle comunità con quelli attesi in una rete casuale. Altri metodi includono le **Tecniche Spettrali**, che utilizzano gli autovettori di matrici associate al grafo (come la matrice Laplaciana) per la clusterizzazione, e gli **Algoritmi basati sulle Clique**, che identificano e analizzano le sovrapposizioni di sottografi altamente connessi.



# Il Concetto di Modularità ($M$)

La **Modularità** ($M$) è una metrica scalare utilizzata nella network analysis per **quantificare la forza della divisione di una rete in moduli o comunità**. In termini semplici, misura quanto bene la rete è stata partizionata in gruppi rispetto a quanto ci si aspetterebbe da una rete casuale con la stessa distribuzione dei gradi. Un valore di modularità elevato (vicino a +1) indica che la partizione della rete in comunità è molto buona, con connessioni **dense all'interno dei gruppi** (connessioni intra-comunità) e connessioni **sparse tra i gruppi** (connessioni inter-comunità). Un valore vicino allo zero o negativo significa che la struttura della rete non è migliore di quella che si avrebbe in una rete casuale, o che la partizione proposta è inefficace.

Matematicamente, la modularità è definita come la differenza tra la frazione di archi che cadono all'interno delle comunità meno la frazione attesa di tali archi, basata su un **modello nullo casuale** (*random null model*). L'obiettivo della maggior parte degli algoritmi di *Community Detection*, come l'Algoritmo di Louvain, è quello di **massimizzare** questa funzione per trovare la partizione ottima della rete. La formula generale della modularità è: $$M = \frac{1}{2m} \sum_{i,j} \left[ A_{ij} - \frac{k_i k_j}{2m} \right] \delta(c_i, c_j)$$ dove $A_{ij}$ è la matrice di adiacenza (che indica se esiste un arco tra il nodo $i$ e $j$), $k_i$ e $k_j$ sono i gradi dei nodi, $m$ è il numero totale di archi della rete, e $\delta(c_i, c_j)$ è un indicatore che vale 1 se i nodi $i$ e $j$ appartengono alla stessa comunità ($c_i = c_j$), e 0 altrimenti. Il termine $A_{ij}$ rappresenta il numero di archi *osservati* tra $i$ e $j$, mentre il termine $\frac{k_i k_j}{2m}$ rappresenta il numero di archi *attesi* tra $i$ e $j$ se la rete fosse casuale.

# Rete ad invarianza di scala

Una rete a invarianza di scala è un tipo di grafo complesso caratterizzato da una distribuzione dei gradi che segue una legge di potenza ($P(k) \sim k^{-\gamma}$), invece della distribuzione di Poisson o Gaussiana tipica delle reti casuali. Questa proprietà matematica fondamentale implica che la rete non ha un grado medio caratteristico che la definisca completamente; al contrario, è dominata da una forte eterogeneità. La conseguenza diretta è l'esistenza di pochi nodi estremamente connessi, noti come hub, che coesistono con la stragrande maggioranza di nodi con pochissime connessioni. Reti reali come il World Wide Web, Internet a livello router, e le reti di interazione proteica mostrano comunemente questa struttura.

Questo tipo di architettura si forma tipicamente attraverso un processo noto come attaccamento preferenziale, descritto dal modello Barabási-Albert (BA). In questo modello, la rete cresce con l'aggiunta continua di nuovi nodi che, con maggiore probabilità, si connettono ai nodi che sono già ricchi di collegamenti (il principio "il ricco diventa più ricco"). Strutturalmente, le reti a invarianza di scala sono estremamente robuste ai guasti casuali (la rimozione di un nodo scelto a caso raramente colpisce un hub), ma sono al contempo molto vulnerabili agli attacchi mirati. La rimozione strategica di anche solo pochi hub critici può infatti frammentare rapidamente e disconnettere vaste porzioni della rete.

# Reti orientate e spazializzazione di una rete

Una rete orientata (o grafo diretto) è un modello di rete in cui la relazione tra i nodi ha una direzione specifica, indicata dagli archi sotto forma di frecce. A differenza delle reti non orientate, dove le connessioni sono simmetriche, un collegamento in un senso (da A a B) non implica automaticamente l'esistenza di un collegamento nel senso opposto (da B ad A). Questa asimmetria è cruciale per modellare sistemi con flussi unidirezionali, come le catene di citazioni accademiche, le interazioni follower-following sui social media, o le dipendenze causali. Nelle reti orientate, il grado di un nodo viene scomposto in in-degree (il numero di connessioni in entrata) e out-degree (il numero di connessioni in uscita), misure che forniscono informazioni distinte sul ruolo del nodo come fonte o destinatario di influenza o flusso.

La spazializzazione in network analysis riguarda l'integrazione della dimensione geografica o fisica dei nodi all'interno della struttura della rete. I nodi non sono trattati come entità astratte, ma sono associati a coordinate spaziali che permettono di misurare la distanza fisica tra loro. Questa distanza diventa un fattore determinante che influenza la formazione o la forza dei collegamenti, un fenomeno noto come decadimento con la distanza (distance decay). L'analisi di reti spazializzate (come reti di trasporto, infrastrutture elettriche o interazioni sociali vincolate dalla prossimità) è fondamentale per comprendere come i vincoli geografici modellino la topologia e la dinamica dei flussi, distinguendo le reti in cui la connettività è limitata dalla distanza fisica rispetto alle reti aspaziali tipiche della comunicazione digitale.

# Ricevute bancarie e filiere produttive

La Ricevuta Bancaria (Ri.Ba) è uno strumento di incasso molto diffuso nel settore commerciale italiano, tramite il quale un creditore (fornitore) affida alla propria banca l'incarico di riscuotere un credito nei confronti di un debitore (cliente) entro una data di scadenza prestabilita. Il processo coinvolge le banche dei due soggetti e utilizza un circuito interbancario per la gestione e l'avviso di pagamento. La Ri.Ba rappresenta il credito di fornitura e, sebbene sia un mezzo di pagamento, è anche uno strumento cruciale per la gestione della liquidità aziendale. Infatti, il creditore può richiedere alla banca l'anticipo dell'importo prima della scadenza (salvo buon fine), utilizzando così la Ri.Ba come garanzia per ottenere fondi immediatamente, trasferendo temporaneamente l'onere dell'incasso all'istituto finanziario.

Disponendo di tutte le ricevute bancarie dell'economia italiana, si potrebbe costruire una rete orientata e pesata estremamente informativa. In questa rete, ogni azienda agirebbe da nodo, e ogni Ri.Ba (o l'insieme delle Ri.Ba) definirebbe un arco orientato che va dal creditore al debitore, rappresentando la direzione del credito commerciale. Il peso dell'arco sarebbe dato dall'importo o dal volume delle transazioni. L'applicazione delle tecniche di network analysis a questo grafo permetterebbe di identificare gli hub finanziari (aziende con elevati flussi di credito), mappare le filiere produttive tramite la community detection e, soprattutto, valutare la vulnerabilità del sistema. Analizzando i percorsi di credito, si potrebbe simulare l'effetto domino che l'insolvenza di un'azienda chiave (un hub) avrebbe sulla liquidità e la stabilità dei suoi creditori e debitori a valle.

# Algoritmo di Louvain

L'algoritmo di Louvain è un metodo euristico greedy ampiamente utilizzato per ottimizzare la modularità ($M$) di una partizione della rete. È noto per la sua efficienza computazionale e la capacità di gestire reti molto grandi. L'algoritmo procede in due fasi principali, ripetute iterativamente: Fase 1 (Ottimizzazione Locale): Ogni nodo è inizialmente considerato una comunità a sé stante. Per ogni nodo, l'algoritmo calcola il guadagno di modularità che si otterrebbe spostando il nodo in una comunità adiacente. Il nodo viene spostato nella comunità che offre il maggiore guadagno positivo. Questa fase si ripete finché non si verificano più spostamenti che migliorino la modularità. Fase 2 (Aggregazione): Viene creata una nuova rete in cui le comunità identificate nella Fase 1 vengono trattate come super-nodi. Gli archi tra i super-nodi rappresentano le connessioni aggregate tra le comunità precedenti, e i loop rappresentano le connessioni interne. L'intero processo (Fase 1 e Fase 2) viene quindi ripetuto sulla rete compressa fino a quando non si verificano ulteriori miglioramenti significativi della modularità, portando a una partizione gerarchica della rete.

# Algoritmo Spinglass

L'algoritmo Spinglass (o Spin-glass model) è un metodo di rilevazione di comunità che trae ispirazione dalla fisica statistica, in particolare dai modelli di vetro di spin e dal simulated annealing. Questo approccio trasforma il problema di trovare le comunità in un problema di minimizzazione dell'energia. I nodi della rete vengono visti come particelle di spin (che possono avere un orientamento discreto, ad esempio $\pm 1$ o un numero intero di stati). Il modello definisce una funzione energetica (Hamiltonian) in cui una configurazione di spin a bassa energia corrisponde a una buona partizione in comunità (i nodi della stessa comunità hanno spin allineati, mentre quelli di comunità diverse hanno spin disallineati). L'algoritmo di simulated annealing viene quindi utilizzato per esplorare lo spazio delle configurazioni: partendo da una "temperatura" alta che permette movimenti casuali, l'algoritmo raffredda gradualmente il sistema, bloccando gli spin in configurazioni a bassa energia che rappresentano le comunità finali. A differenza dei metodi greedy, Spinglass è in grado di sfuggire agli ottimi locali grazie all'elemento di casualità iniziale, anche se è computazionalmente più intensivo e più lento su reti molto grandi.

# Algoritmi alternativi

**Algoritmi Divisivi e Aggregativi Basati sulla Connettività**

Un'alternativa importante ai metodi di ottimizzazione della modularità è rappresentata dagli algoritmi divisivi, come l'algoritmo Girvan-Newman. Questo approccio opera in modo opposto rispetto ai metodi aggregativi, iniziando con la rete completa e rimuovendo iterativamente gli archi più probabili che collegano diverse comunità. L'importanza di un arco viene determinata calcolando la sua Betweenness Centrality (centralità di intermediazione): gli archi che si trovano sul maggior numero di cammini minimi tra tutte le coppie di nodi sono considerati ponti inter-comunità e vengono eliminati per primi. Il processo continua fino a quando non si raggiunge la partizione con la modularità massima. Sebbene offra risultati chiari, il metodo Girvan-Newman è noto per essere computazionalmente oneroso, risultando poco pratico per le reti di grandi dimensioni.

**Algoritmi per Comunità Sovrapposte e Inferenza Statistica**  

Un'altra categoria di algoritmi si concentra sulla realtà delle comunità sovrapposte, dove un singolo nodo può appartenere a più gruppi, come tipicamente avviene nei social network (un utente appartiene sia a un gruppo familiare che a un gruppo lavorativo). Il Clique Percolation Method (CPM) è un esempio di questo approccio, che identifica le comunità come l'unione di k-clique (sottografi completi di dimensione $k$) che si intersecano o "percolano" l'una nell'altra. In contrasto con questi metodi basati sulla topologia, esistono gli algoritmi di inferenza statistica, come lo Stochastic Block Model (SBM). L'SBM inquadra il problema come la ricerca del modello statistico più probabile che abbia generato la rete osservata, suddividendo i nodi in "blocchi" con specifiche probabilità di connessione interna ed esterna. Questo offre una base matematica più rigorosa rispetto ai semplici metodi di ottimizzazione.

**Algoritmi Basati sulla Propagazione**

Infine, l'Algoritmo di Propagazione delle Etichette (LPA) offre una soluzione estremamente rapida ed efficiente. In questo metodo, a ogni nodo viene inizialmente assegnata un'etichetta di comunità unica. Ad ogni step, ciascun nodo adotta l'etichetta più diffusa tra i suoi vicini. Questo processo di diffusione locale si ripete finché le etichette non convergono, identificando rapidamente i gruppi. L'LPA è apprezzato per la sua velocità (quasi lineare nel numero di archi) e per il fatto che non richiede la pre-impostazione di parametri (come il numero di comunità). Il suo principale svantaggio è la natura non deterministica, in quanto l'ordine in cui i nodi aggiornano le loro etichette può influire sul risultato finale della partizione.

# Newman (2026), Modularity and community structure in networks

Il testo presenta una metodologia per identificare la struttura comunitaria all'interno di sistemi complessi che possono essere modellati come reti, inclusi esempi sociali, informatici e biologici. L'autore definisce questo problema come la massimizzazione di una funzione statistica chiamata "modularità," che misura la densità dei collegamenti all'interno dei gruppi rispetto a quanto atteso casualmente. L'innovazione principale consiste nello sviluppare un algoritmo spettrale basato sull'espressione della modularità in funzione degli autovettori di una matrice specifica per la rete, denominata "matrice di modularità." Questo metodo consente di suddividere la rete gerarchicamente, stabilendo che la divisione si interrompa quando un gruppo diventa indivisibile, ossia non può essere scisso ottenendo un guadagno positivo di modularità. L'approccio, sebbene talvolta raffinato tramite una ottimizzazione di precisione (fine-tuning), si dimostra superiore in termini di qualità dei risultati e di velocità computazionale rispetto ai principali algoritmi preesistenti. Le applicazioni a reti reali, come le collaborazioni scientifiche e i social network politici, confermano la capacità del metodo di estrarre raggruppamenti significativi dai dati.

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from community import community_louvain # Necessita l'installazione: pip install python-louvain

# --- 1. Generazione del Dataset Simulato ---

# 10 settori aziendali di esempio
settori = [
    'Manifattura', 'Materie Prime', 'Trasporti & Logistica',
    'Distribuzione', 'Retail', 'Servizi IT',
    'Finanza', 'Agricoltura', 'Energia', 'Consulenza'
]

# Numero totale di transazioni simulate (fatture)
num_transazioni = 1000

# Creazione di transazioni casuali
np.random.seed(42)
data = {
    'Settore_Creditore': np.random.choice(settori, num_transazioni),
    'Settore_Debitore': np.random.choice(settori, num_transazioni),
    'Importo': np.random.randint(100, 50000, num_transazioni),
    'Connessioni': 1 # Ogni riga rappresenta 1 connessione
}

df_transazioni = pd.DataFrame(data)

# Rimuovi le transazioni di un settore verso sé stesso (opzionale, ma tipico in una filiera)
df_transazioni = df_transazioni[df_transazioni['Settore_Creditore'] != df_transazioni['Settore_Debitore']]

# Aggregazione: sommiamo gli importi e contiamo le connessioni per ogni coppia unica Creditore -> Debitore
df_network = df_transazioni.groupby(['Settore_Creditore', 'Settore_Debitore']).agg(
    Numero_Connessioni=('Connessioni', 'sum'),
    Somma_Importi=('Importo', 'sum')
).reset_index()

# Visualizzazione del DataFrame aggregato
print("--- DataFrame Aggregato (Transazioni per Coppia di Settori) ---")
print(df_network.head())
print(f"\nCoppie di relazioni uniche: {len(df_network)}")

--- DataFrame Aggregato (Transazioni per Coppia di Settori) ---
  Settore_Creditore Settore_Debitore  Numero_Connessioni  Somma_Importi
0       Agricoltura       Consulenza                  14         282835
1       Agricoltura    Distribuzione                   4         128662
2       Agricoltura          Energia                  12         240489
3       Agricoltura          Finanza                   9         261515
4       Agricoltura      Manifattura                  10         272158

Coppie di relazioni uniche: 90


In [ ]:
# --- 2. Costruzione del Grafo (NetworkX) ---

# Creazione del grafo diretto (DiGraph) poiché la transazione ha una direzione
G = nx.DiGraph()

# Aggiunta degli archi (edges) dal DataFrame
# Usiamo 'Numero_Connessioni' come peso (weight) dell'arco
for index, row in df_network.iterrows():
    G.add_edge(
        row['Settore_Creditore'], # Nodo Sorgente
        row['Settore_Debitore'], # Nodo Destinazione
        weight=row['Numero_Connessioni'], # Peso: Numero di fatture
        amount=row['Somma_Importi']      # Attributo aggiuntivo: Somma degli importi
    )

print("\n--- Informazioni di Base sul Grafo ---")
print(f"Numero di Nodi (Settori): {G.number_of_nodes()}")
print(f"Numero di Archi (Relazioni uniche): {G.number_of_edges()}")

# Esempio di calcolo di una metrica: Centralità di Grado (Degree Centrality)
# Questa misura identifica i settori con più partner commerciali (in entrata e in uscita)
degree_centrality = nx.degree_centrality(G)
# Ordina i settori per centralità
top_centrality = sorted(degree_centrality.items(), key=lambda item: item[1], reverse=True)

print("\n--- Centralità di Grado (Top 3 Settori) ---")
for sector, centrality in top_centrality[:3]:
    print(f"Settore: {sector}, Centralità: {centrality:.4f}")


--- Informazioni di Base sul Grafo ---
Numero di Nodi (Settori): 10
Numero di Archi (Relazioni uniche): 90

--- Centralità di Grado (Top 3 Settori) ---
Settore: Agricoltura, Centralità: 2.0000
Settore: Consulenza, Centralità: 2.0000
Settore: Distribuzione, Centralità: 2.0000


In [ ]:
# --- 3. Community Detection (Algoritmo di Louvain) ---

# Nota: Louvain lavora meglio sui grafi non diretti, quindi creiamo una versione non direzionata (Graph)
# Manteniamo i pesi (Numero_Connessioni)
G_undirected = G.to_undirected()

# Trova la partizione (le comunità) utilizzando il peso degli archi
# Utilizziamo la funzione `best_partition` della libreria python-louvain
# La variabile `partition` è un dizionario: {nodo: ID_comunità}
partition = community_louvain.best_partition(G_undirected, weight='weight')
modularity = community_louvain.modularity(partition, G_undirected, weight='weight')

# Raggruppamento dei settori per comunità (filiere)
filiere = {}
for nodo, id_comunita in partition.items():
    if id_comunita not in filiere:
        filiere[id_comunita] = []
    filiere[id_comunita].append(nodo)

print("\n--- Risultato della Community Detection (Filiere Emerse) ---")
print(f"Modularity del network (qualità della divisione): {modularity:.4f}")
print(f"Numero di Filiere (Comunità) identificate: {len(filiere)}")

# Stampa le filiere identificate
for id_comunita, settori_comunita in filiere.items():
    print(f"\nFiliere {id_comunita}:")
    print(", ".join(settori_comunita))

# Aggiunta dell'ID di Comunità come attributo a ogni nodo per la visualizzazione futura
for nodo, id_comunita in partition.items():
    G.nodes[nodo]['filiere_id'] = id_comunita


--- Risultato della Community Detection (Filiere Emerse) ---
Modularity del network (qualità della divisione): 0.0000
Numero di Filiere (Comunità) identificate: 1

Filiere 0:
Agricoltura, Consulenza, Distribuzione, Energia, Finanza, Manifattura, Materie Prime, Retail, Servizi IT, Trasporti & Logistica
